## Mengimpor Library

Sebelum memulai pemodelan, kita perlu mengimpor library-library yang diperlukan:
- **pandas**: untuk mengolah data dalam bentuk DataFrame
- **numpy**: untuk operasi matematika 
- **kagglehub**: untuk mengunduh dataset dari Kaggle

In [3]:
# Kode

import pandas as pd
import numpy as np

!pip install kagglehub

import kagglehub as kaggle

!pip show kagglehub

Name: kagglehub
Version: 0.3.13
Summary: Access Kaggle resources anywhere
Home-page: https://github.com/Kaggle/kagglehub
Author: 
Author-email: Kaggle <support@kaggle.com>
License: Apache License
                                   Version 2.0, January 2004
                                http://www.apache.org/licenses/
        
           TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION
        
           1. Definitions.
        
              "License" shall mean the terms and conditions for use, reproduction,
              and distribution as defined by Sections 1 through 9 of this document.
        
              "Licensor" shall mean the copyright owner or entity authorized by
              the copyright owner that is granting the License.
        
              "Legal Entity" shall mean the union of the acting entity and all
              other entities that control, are controlled by, or are under common
              control with that entity. For the purposes of thi

## Data Preparation

Tahap awal dalam machine learning adalah mempersiapkan data yang akan digunakan untuk pemodelan. Pada bagian ini kita akan muat, eksplorasi data secara sederhana, dan periksa apakah ada data yang hilang atau duplikat.

### Unduh dan Muat Data

Kita akan menggunakan dataset California Housing Prices dari kaggle. Dataset ini berisi informasi tentang harga rumah di California berdasarkan berbagai fitur seperti lokasi, umur rumah, jumlah kamar, dan karakteristik demografis. Kalian bisa memerika informasi lebih lanjut dari dataset pada [kaggle.com/datasets/camnugent/california-housing-prices](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Proses pengunduhan dataset dilakukan menggunakan `kagglehub.dataset_download()` yang akan mengunduh dan menyimpan dataset ke direktori lokal. Setelah mendapatkan path direktori, kita menggunakan [f-string](https://www.w3schools.com/python/python_string_formatting.asp) (`f"{path}/housing.csv"`) untuk menggabungkan path dengan nama file dataset yang sudah diketahui dari Kaggle yaitu `housing.csv`. Data kemudian dibaca menggunakan `pd.read_csv()` dan disimpan dalam DataFrame untuk analisis lebih lanjut.

In [4]:
# Kode

kaggle.dataset_download("camnugent/california-housing-prices")

'C:\\Users\\Arcleid\\.cache\\kagglehub\\datasets\\camnugent\\california-housing-prices\\versions\\1'

In [5]:
path_housing = f"C:\\Users\\Arcleid\\.cache\\kagglehub\\datasets\\camnugent\\california-housing-prices\\versions\\1\housing.csv"
df_house = pd.read_csv(path_housing)
df_house

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


Kolom yang akan kita gunakan sebagai target untuk memprediksi harga rumah adalah `median_house_value`. 

### Exploratory Data Analysis

Pada tahap ini kita akan melakukan EDA sederhana dengan memeriksa informasi umum dan periksa apakah ada nilai yang kosong atau duplikasi ini. Pertama-tama, mulai dari melihat informasi umum dataset yang memberi tahu jumlah dan tipe data menggunakan fungsi `info()` dari pandas

In [6]:
# Kode

df_house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


Lanjut memeriksa fitur dengan nilai yang kosong menggunakan fungsi `.isnull()` dari pandas, lalu bisa kita lakukan [method chaining](https://www.geeksforgeeks.org/python/method-chaining-in-python/) dengan fungsi `.sum()` .

In [7]:
# Kode

df_house.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Terakhir kita bisa memeriksa apakah ada nilai duplikat pada dataset dengan fungsi `.duplicated()` yang dikombinasikan dengan fungsi `.sum()`.

In [8]:
# Kode

df_house.duplicated().sum()

0

## Data Splitting

Sebelum masuk ke tahap preprocessing untuk melatih model, kita perlu segera membagi dataset menjadi training set untuk pelatihan, validation set untuk refinement, dan testing set untuk pengujian akhir model. Di sini kita menggunakan pembagian dengan 60/20/20.

<img src="https://i.sstatic.net/FnY8G.png" alt="Data Splitting Diagram">

### Shuffling

Pembagian data perlu dilakukan dengan shuffling (pengacakan) untuk memastikan bahwa data terdistribusi secara acak di setiap split. Proses ini bisa dilakukan menggunakan numpy. Di sini digunakan pembagian pada DataFrame sebanyak:
- 20% untuk validation set
- 20% untuk test set
- 60% sisanya untuk training set

In [9]:
# Kode

valdt_set_house = df_house.sample(frac=0.2)
valdt_set_house

testn_set_house = df_house.sample(frac=0.2)
testn_set_house

train_set_house = df_house.sample(frac=0.6)
train_set_house

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
8406,-118.36,33.94,33.0,939.0,284.0,1309.0,250.0,3.4063,152300.0,<1H OCEAN
5361,-118.41,34.04,52.0,2113.0,332.0,800.0,327.0,11.1768,500001.0,<1H OCEAN
15180,-117.08,33.03,18.0,1339.0,284.0,761.0,290.0,5.3074,137200.0,<1H OCEAN
4568,-118.26,34.06,42.0,2541.0,1282.0,3974.0,1189.0,1.5854,87500.0,<1H OCEAN
20368,-118.91,34.18,17.0,3220.0,716.0,1381.0,733.0,2.8958,176000.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
19392,-120.85,37.77,35.0,404.0,96.0,261.0,100.0,2.4583,75000.0,INLAND
12397,-116.30,33.68,10.0,2387.0,481.0,863.0,304.0,2.8882,137500.0,INLAND
7401,-118.24,33.96,37.0,1602.0,388.0,1553.0,342.0,2.0655,93400.0,<1H OCEAN
7513,-118.26,33.90,38.0,1566.0,318.0,981.0,318.0,4.0234,111900.0,<1H OCEAN


### Ekstraksi Target Variable

Pada tahap ini kita memisahkan target variable (median_house_value) ekstraksi target dari dataset sebagai numpy array untuk diproses model nanti. Setelah ekstraksi, kolom target dihapus dari DataFrame fitur untuk menghindari [data leakage](https://www.kaggle.com/code/alexisbcook/data-leakage).

In [10]:
# Kode

house_mhv_array = df_house["median_house_value"].values

In [11]:
type(house_mhv_array)
house_mhv_array

array([452600., 358500., 352100., ...,  92300.,  84700.,  89400.])

In [12]:
df_house = df_house.drop("median_house_value", axis=1)
df_house

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND


## Pre-processing

Tahap preprocessing adalah langkah krusial untuk mempersiapkan data agar bisa digunakan oleh algoritma machine learning. 

### Missing Values Handling

Dataset ini memiliki beberapa missing values pada kolom 'total_bedrooms'. Nilai yang hilang ini memiliki tipe data `null` atau `NaN` yang akan menimbulkan error ketika diproses. Untuk menangani ini ada beberapa cara, masing-masing dengan kelebihan dan kekurangan, tapi untuk menyerdehanakan, kita akan mengisinya dengan 0.

In [13]:
# Kode

df_house["total_bedrooms"] = df_house["total_bedrooms"].fillna(0)
df_house["total_bedrooms"].isnull().sum()

0

### Encoding

Variabel kategorikal 'ocean_proximity' perlu diubah menjadi format numerik agar bisa diproses oleh model machine learning. Ingat bahwa model machine learning seperti regresi linear pada dasarnya adalah algoritma yang melakukan kalkulasi matematika, dan tentunya tidak bisa menghitung nilai yang bukan angka. Ada banyak metode konversi variabel kategorikal ke numerik, salah satunya adalah **One-Hot Encoding** yang membuat kolom binary terpisah untuk setiap kategori. 

<img src="https://miro.medium.com/1*ggtP4a5YaRx6l09KQaYOnw.png" alt="One-Hot Encoding" width="80%">

In [14]:
# Kode

# Manual, pd_get_dummies

#encoder = pd.get_dummies(df_house["ocean_proximity"])
#df_house_encode = pd.concat([df_house, encoder], axis=1)
#df_house_encode

# Import dari sklearn.preprocessing.OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, dtype=int)
encoded = encoder.fit_transform(df_house[["ocean_proximity"]])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(["ocean_proximity"]))
encoded_df.head()

df_house_encoded = pd.concat([df_house.reset_index(drop=True), encoded_df], axis=1)
df_house_encoded.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY,0,0,0,1,0


In [15]:
#Manual

# #df_house_encode["<1H OCEAN"] = df_house_encode["<1H OCEAN"].astype(int)
# #df_house_encode["INLAND"] = df_house_encode["INLAND"].astype(int)
# df_house_encode["ISLAND"] = df_house_encode["ISLAND"].astype(int)
# df_house_encode["NEAR BAY"] = df_house_encode["NEAR BAY"].astype(int)
# df_house_encode["NEAR OCEAN"] = df_house_encode["NEAR OCEAN"].astype(int)

# df_house_encode

NameError: name 'df_house_encode' is not defined

In [17]:
#df_house = df_house_encode
df_house = df_house_encoded

### Membuat Fitur Baru

Salah satu proses feature engineering adalah membuat fitur baru yang lebih informatif dari data yang sudah ada. Di sini kita akan bereksperimen membuat fitur baru yang lebih bermakna. Misalnya, daripada total kamar atau kamar tidur saja, lebih berguna melihat rasio seperti kamar per rumah tangga, perbandingan kamar tidur terhadap kamar, atau populasi per rumah tangga.

In [18]:
# Kode

df_house["bedrooms_per_household"] = df_house["total_bedrooms"] / df_house["households"]

df_house["ratio_bedroom_and_rooms"] = df_house["total_bedrooms"] / df_house["total_rooms"]

df_house["population_per_household"] = df_house["population"] / df_house["households"]

df_house["rooms_per_household"] = df_house["total_rooms"] / df_house["households"]

df_house

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN,bedrooms_per_household,ratio_bedroom_and_rooms,population_per_household,rooms_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,0,0,0,1,0,1.023810,0.146591,2.555556,6.984127
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY,0,0,0,1,0,0.971880,0.155797,2.109842,6.238137
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY,0,0,0,1,0,1.073446,0.129516,2.802260,8.288136
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY,0,0,0,1,0,1.073059,0.184458,2.547945,5.817352
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY,0,0,0,1,0,1.081081,0.172096,2.181467,6.281853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,INLAND,0,1,0,0,0,1.133333,0.224625,2.560606,5.045455
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,INLAND,0,1,0,0,0,1.315789,0.215208,3.122807,6.114035
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,INLAND,0,1,0,0,0,1.120092,0.215173,2.325635,5.205543
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,INLAND,0,1,0,0,0,1.171920,0.219892,2.123209,5.329513


Untuk memeriksa apakah fitur baru ini bermakna atau tidak bisa dilakukan melalu perhitungan korelasi, yakni ukuran seberapa kuat dan seberapa arah hubungan antara dua variabel secara linear (positif ke atas atau negatif ke bawah). Korelasi ini dihitung menggunakan [koefisien korelasi Pearson](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient). Pandas menyediakan kalkulasi ini melalui fungsi `.corr()`. Dari fungsi `.corr()` bisa kita filter untuk melihat hubungannya dengan `median_house_value` saja sebagai target dan sorting secara descending menggunakan fungsi `.sort_values(ascending=False)`

In [20]:
df_house.info()
df_house = df_house.drop("ocean_proximity", axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitude                   20640 non-null  float64
 1   latitude                    20640 non-null  float64
 2   housing_median_age          20640 non-null  float64
 3   total_rooms                 20640 non-null  float64
 4   total_bedrooms              20640 non-null  float64
 5   population                  20640 non-null  float64
 6   households                  20640 non-null  float64
 7   median_income               20640 non-null  float64
 8   ocean_proximity             20640 non-null  object 
 9   ocean_proximity_<1H OCEAN   20640 non-null  int32  
 10  ocean_proximity_INLAND      20640 non-null  int32  
 11  ocean_proximity_ISLAND      20640 non-null  int32  
 12  ocean_proximity_NEAR BAY    20640 non-null  int32  
 13  ocean_proximity_NEAR OCEAN  206

In [23]:
# Kode

df_mhv = pd.Series(house_mhv_array, name="median_house_value")

df_house_corr = df_house.corrwith(df_mhv).sort_values(ascending=False)

df_house_corr

median_income                 0.688075
ocean_proximity_<1H OCEAN     0.256617
ocean_proximity_NEAR BAY      0.160284
rooms_per_household           0.151948
ocean_proximity_NEAR OCEAN    0.141862
total_rooms                   0.134153
housing_median_age            0.105623
households                    0.065843
total_bedrooms                0.049148
ocean_proximity_ISLAND        0.023416
population_per_household     -0.023737
population                   -0.024650
bedrooms_per_household       -0.045163
longitude                    -0.045967
latitude                     -0.144160
ratio_bedroom_and_rooms      -0.238759
ocean_proximity_INLAND       -0.484859
dtype: float64

Koefisen korelasi ini memiliki rentang -1 sampai 1. Semakin dekat dengan 1, maka terdapat korelasi positif. Ketika semakin dekat dengan -1, maka terdapat korelasi negatif. Apabila mendekati 0, maka tidak terdapat korelasi yang linear. Terlihat bahwa `median_income` mempunyai korelasi positif yang paling kuat. Dari tiga fitur baru yang dibuat, dua di antaranya, yaitu `rooms_per_house` punya korelasi positif dan `bedrooms_ratio` punya korelasi negatif yang cukup tinggi jika dibandingkan dengan fitur lainnya.

### Fungsi Preprocessing

Perhatikan bahwa, proses preprocessing sebelumnya hanya dilakukan pada DataFrame sebelum splitting. Untuk memastikan konsistensi, kita dapat membuat fungsi `prepare_X()` yang mengkombinasikan semua langkah preprocessing dalam satu tempat pada fitur dari training, validation, dan test set nanti.

In [41]:
# Kode

def preparation_X(df: pd.DataFrame):
    df = df.copy()
    
    target = df["median_house_value"].to_numpy()
    
    df = df.drop(columns=["median_house_value"])
    
    df.isnull().sum()
    
    df = df.fillna(0)

    encoder = OneHotEncoder(sparse_output=False, dtype=int)
    encoded = encoder.fit_transform(df[["ocean_proximity"]])
    encoded_df = pd.DataFrame(
        encoded, 
        columns=encoder.get_feature_names_out(["ocean_proximity"]),
        index=df.index
    )

    df = pd.concat([df.drop(columns=["ocean_proximity"]), encoded_df], axis=1)
    
    df = df.loc[:, ~df.columns.duplicated()]
    
    return df, target

## Modeling

Pada bagian ini kita akan membuat model dengan algoritma regresi linear secara manual dibantu numpy. Diketahui bahwa persamaan atau rumus regresi linear untuk satu fitur adalah sebagai berikut:


 <img src="https://lh3.googleusercontent.com/d/1kpIila4WGjbvK2ind5sttFI_qRzOiUnh" alt="rumus-regresi-linear-satu-fitur.png" width="500"/>

### Regresi Linear

Diketahui bahwa **Normal Equation** atau persamaan normal dapat digunakan untuk mendapatkan nilai bias $w_{0}$ dan bobot $w$ selama pelatihan sebagai berikut:

 <img src="https://lh3.googleusercontent.com/d/11TLCxNQuoCDfo6iFy-laOgszod_EHl1m" alt="normal-equation.png" width="300"/>

Sebelum menerjemahkan ke NumPy, pertama perlu diketahui bahwa matrix $X$ hanya akan berisi input fitur-fitur dari dataset atau $x_{i1}$, $x_{i2}$, hingga $x_{in}$. Untuk mendapatkan $w_0$, kita perlu menambahkan fitur $x_{i0}$ yang selalu bernilai 1. Cara untuk menambahkan ini adalah dengan fungsi `column_stack` pada X sebelum menghitung $X^T$.
```python
np.column_stack([ones, X])
```
Dimana `ones` adalah array NumPy berisi angka 1 dengan panjang sama dengan jumlah sampel dalam X.

 <img src="https://lh3.googleusercontent.com/d/1Ff4tFPIgUuNIfO78vPBj1Qv-bRWOQST6" alt="reguralisasi.png" width="300"/>

Selanjutnya kita bisa menerjemahkan persamaan ke NumPy sebagai berikut:

* $X^T$ artinya transpose dari $X$. Di NumPy ditulis `X.T`.
* $X^T X$ artinya perkalian matriks, bisa dilakukan dengan `X.T.dot(X)`.
* $X^{-1}$ artinya invers dari $X$. Untuk menghitungnya bisa pakai `np.linalg.inv`.

Jadi rumus tadi bisa langsung ditulis sebagai:

```python
inv(X.T.dot(X)).dot(X.T).dot(y)
```

Akan tetapi, untuk menghindari permasalahan numerical instability, normal equation membutuhkan penambahan reguralisasi sebagai berikut:

 <img src="https://lh3.googleusercontent.com/d/1Y5XbrXo-rLUqcs-qtvEmZKn2Paowhpps" alt="reguralisasi.png" width="300"/>

Terlihat bahwa dalam menghitung invers matriks $(X^T X)^{-1}$, perlu ditambahkan perkalian bilangan $\alpha$ dengan matriks identitas $I$ ke $X^T X$ sehingga menjadi $(X^T X + \alpha I)^{-1}$. Dengan cara ini, semua angka satu di diagonal $I$ menjadi $\alpha$. Lalu kita jumlahkan $\alpha I$ dengan $X^T X$, yang berarti menambahkan $\alpha$ ke semua elemen diagonal dari $X^T X$.

Rumus ini dapat langsung diterjemahkan ke dalam kode NumPy, misal dengan $\alpha$ bernilai 0.01:

```python
XTX = X_train.T.dot(X_train)
XTX = XTX + 0.01 * np.eye(XTX.shape[0])
```

Fungsi `np.eye` membuat array NumPy dua dimensi yang juga merupakan matriks identitas berukuran salah satu sampel $X^T X$ yakni di index paling pertama (`XTX.shape[0]`). Ketika kita mengalikannya dengan 0.01, angka satu pada diagonal utama menjadi 0.01. Jadi, ketika kita menambahkan matriks ini ke $X^T X$, kita hanya menambahkan 0.01 pada diagonal utamanya.

 <img src="https://lh3.googleusercontent.com/d/1Q0Cj6EmkdCH-UK8a0wdDBVBG4CzE-K3J" alt="one-eye.png" width="300"/>

Oleh karena itu, langkah untuk mengimplementasikan Normal Equation dengan NumPy adalah:
1. Buat fungsi yang menerima matriks fitur **X**, target **y**, dan parameter regularisasi `r`.
2. Tambahkan satu kolom dummy berisi 1 ke **X** untuk bias $w_0$.
3. Hitung $X^T X$, lalu tambahkan `r * I` ke diagonal utamanya.
4. Hitung invers dan bobot: `w = inv(XTX).dot(X.T).dot(y)`.
5. Pisahkan `w` menjadi bias `w_0` (elemen pertama) dan bobot fitur lainnya, lalu kembalikan hasilnya.

Mari kita implementasikan.

In [45]:
# Kode

def normal_equation(X, y, r=0.0):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    
    XTX = X_b.T.dot(X_b)
    reg = r * np.eye(XTX.shape[0])
    reg[0, 0] = 0
    XTX_r = XTX + reg
    
    w = np.linalg.inv(XTX_r).dot(X_b.T).dot(y)
    
    w0 = w[0]
    weights = w[1:]
    
    return w0, weights

Parameter tambahan `r` mengontrol seberapa besar regularisasi. Parameter ini sama dengan bilangan $\alpha$ dalam rumus yang kita tambahkan ke diagonal utama dari $X^T X$. Selanjutnya coba kita berikan input X dan output y dari training set untuk mendapatkan bias dan bobotnya. Sementara kita gunakan kekuatan reguralisasi `r` sebesar 1.

In [1]:
df_house

NameError: name 'df_house' is not defined

In [27]:
# Kode

X = df_house.values.astype(float)

y = house_mhv_array.astype(float)

w0, weights = normal_equation(X, y, r=1)

weights_df = pd.DataFrame({
    "feature": df_house.columns,
    "weight": weights
})

print("Bias (w0):", w0)
print(weights_df)

Bias (w0): -2396503.0336709116
                       feature         weight
0                    longitude  -27592.007112
1                     latitude  -26333.150876
2           housing_median_age    1078.826763
3                  total_rooms       1.458734
4               total_bedrooms      -8.175813
5                   population     -42.190296
6                   households     138.083940
7                median_income   38800.437894
8    ocean_proximity_<1H OCEAN  -19268.819684
9       ocean_proximity_INLAND  -55797.478071
10      ocean_proximity_ISLAND  113021.481730
11    ocean_proximity_NEAR BAY  -23195.495667
12  ocean_proximity_NEAR OCEAN  -14759.688315
13      bedrooms_per_household  -28189.245042
14     ratio_bedroom_and_rooms  314347.687159
15    population_per_household      91.038926
16         rooms_per_household    8805.542177


Bias dan bobot ini akhirnya bisa kita gunakan untuk prediksi. Diketahui bahwa rumus untuk melakukan prediksi regresi linear yang mempunyai banyak fitur adalah sebagai berikut:

 <img src="https://lh3.googleusercontent.com/d/1k0a5UXs6d3Y2bYRsY1hnC1EuR5OS_6t3" alt="prediksi-regresi.png" width="300"/>

Diterjemahkan ke NumPy menjadi:
```python
y_pred = w0 + X.dot(w)
```

Mari kita gunakan untuk prediksi pada training set

In [28]:
# Kode

y_pred = w0 + X.dot(weights)

y_pred

array([405620.12988056, 434230.50614765, 381919.0390698 , ...,
        37128.61852095,  48220.0360339 ,  63125.55232055])

### Evaluasi

Untuk mengevaluasi performa model dalam prediksi, kita menggunakan dua metrik utama:

- **RMSE (Root Mean Square Error)**: mengukur rata-rata error prediksi dalam unit yang sama dengan target variable. Semakin kecil nilainya, semakin baik.

 <img src="https://lh3.googleusercontent.com/d/1imAxrqJUHmRN_cr7-1mJenqPybLy69Pi" alt="rmse.png" width="300"/>

- **R² Score**: mengukur seberapa baik model menjelaskan variance dalam data. Nilai berkisar 0-1, dimana 1 berarti perfect fit.

 <img src="https://lh3.googleusercontent.com/d/1Z-lGdXOpUK3MQJd3aQySNkNJjKuuLnVk" alt="r-squared.png" width="300"/>

Kedua rumus ini bisa sangat mudah untuk dikonversikan ke NumPy

In [29]:
# Kode

rmse = np.sqrt(np.mean((y - y_pred) ** 2))
rmse

67961.97055814057

Mari kita gunakan keduanya pada training dan validation set

In [30]:
# Kode

res = np.sum((y - y_pred) ** 2)
tot = np.sum((y - np.mean(y)) ** 2)
r_square = 1 - res / tot
r_square

0.6531239231547021

### Hyperparameter Tuning

Penambahan parameter `r` untuk mengatur seberapa besar reguralisasi pada fungsi `train_linear_regression()` merupakan bentuk hyperparamater pada model, yakni parameter yang diatur sebelum pelatihan model oleh manusia atau kita sendiri. Oleh karena itu, kita perlu mencoba berbagai nilai parameter yang berbeda untuk mendapatkan hasil terbaik. Proses pengaturan hyperparameter ini disebut dengan hyperparameter tuning.

In [32]:
# Kode

r_values = [0, 0.01, 0.1, 1, 10, 100]
results = []

for r in r_values:
    w0, weights = normal_equation(X, y, r)
    y_pred = w0 + X.dot(weights)
    
    rmse = np.sqrt(np.mean((y - y_pred) ** 2))
    res = np.sum((y - y_pred) ** 2)
    tot = np.sum((y - np.mean(y)) ** 2)
    r_square = 1 - res / tot
    
    results.append((r, rmse, r_square))

tuning_results = pd.DataFrame(results, columns=["r", "RMSE", "R^2"])
print(tuning_results)


        r          RMSE         R^2
0    0.00  2.686412e+06 -540.985328
1    0.01  6.795862e+04    0.653158
2    0.10  6.795866e+04    0.653158
3    1.00  6.796197e+04    0.653124
4   10.00  6.807415e+04    0.651978
5  100.00  6.844913e+04    0.648133


### Evaluasi Model Akhir

Setelah menemukan hyperparameter terbaik yakni `r` di 0.01, kita dapat melatih ulang model final pada kombinasi training dan validastion set lalu mengevaluasi model final pada test set untuk menguji generalisasi dari model.

In [42]:
# Kode

trainval_set_house = pd.concat([train_set_house, valdt_set_house])
X_trainval, y_trainval = preparation_X(trainval_set_house)
w0, weights = normal_equation(X_trainval.to_numpy(), y_trainval, r=0.01)

In [43]:
X_test, y_test = preparation_X(testn_set_house)

## Prediction

Pada tahap akhir, kita menggunakan model terbaik untuk melakukan contoh prediksi pada satu sample test untuk mensimulasikan bagaiman model ini dapat digunakan untuk prediksi pada data baru.

In [44]:
# Kode
y_pred_test = w0 + X_test.to_numpy().dot(weights)

rmse_test = np.sqrt(np.mean((y_test - y_pred_test) ** 2))

res = np.sum((y_test - y_pred_test) ** 2)
tot = np.sum((y_test - np.mean(y_test)) ** 2)
r_square_test = 1 - res / tot

print("Final Evaluation on Test Set:")
print("RMSE:", rmse_test)
print("R^2:", r_square_test)

Final Evaluation on Test Set:
RMSE: 68569.65446793118
R^2: 0.6430893466436727
